In [74]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Scrape data set from wiki website

In [76]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

data = response.text
soup = BeautifulSoup(data,'html.parser')
wiki_table=soup.find('table')

df = pd.read_html(str(wiki_table))[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Drop First column and arrange columns name

In [77]:
df.drop(0,inplace=True)

df.columns = ['PostalCode','Borough','Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"


# Cleaning data as ignoring the rows which the "Borough" contains 'Not assigned'

In [78]:
df2=df[df['Borough'].str.contains("Not assigned") == False].reset_index()
df2.head()

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Combine the rows into one row (and seperate with comma) which PostalCode and Borough as same value

In [79]:
df3= df2.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df3.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Get the number of rows of dataframe



In [80]:
df3.shape

(103, 3)

df3.to_csv('Capstone_part1.csv')
print('Successfully Saved!')

# Part 2 Segmenting & Clustering Toronto

In [81]:
import pandas as pd

# get the data and create the dataframe

In [83]:
url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geo=pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Adjust the column name same as the first dataset

In [84]:
df_geo = df_geo.rename(columns = {'Postal Code':'PostalCode'}) 
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Upload first dataset

In [86]:
df3= df2.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Merge 2 data sets

In [87]:
df3 = pd.merge(df3, df_geo, on = 'PostalCode')
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Number of rows

In [88]:
df3.shape

(103, 5)

# Part 3 Segmenting & Clustering Toronto

In [89]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# First Data Set

In [90]:
df3= df2.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# We work with only "Borough" which contain the word Toronto 

In [91]:
df4=df3[df3['Borough'].str.contains('Toronto')]
df5=df4.reset_index(drop=True)
df5.head()

,PostalCode,Borough,Neighborhood
0,M4E,East Toronto,The Beaches
1,M4K,East Toronto,"The Danforth West, Riverdale"
2,M4L,East Toronto,"India Bazaar, The Beaches West"
3,M4M,East Toronto,Studio District
4,M4N,Central Toronto,Lawrence Park


# Number of rows

In [92]:
df5.shape

(40, 3)

# Learn Different Value of each Burrough

In [93]:
df5['Borough'].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Toronto/York         1
Name: Borough, dtype: int64

# Create a new column as Label and get the date from 'Borough' as integer

In [94]:
df5['Label']=df5['Borough'].replace(to_replace=['Downtown Toronto','Central Toronto','West Toronto','East Toronto'],value=[1,2,3,4],inplace=False)
df5.head()

,PostalCode,Borough,Neighborhood,Label
0,M4E,East Toronto,The Beaches,4
1,M4K,East Toronto,"The Danforth West, Riverdale",4
2,M4L,East Toronto,"India Bazaar, The Beaches West",4
3,M4M,East Toronto,Studio District,4
4,M4N,Central Toronto,Lawrence Park,2


# Coordinates of Toronto

In [104]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [103]:
kclusters=len(df5.Label.unique())


map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, cluster in zip(df5['Latitude'], df5['Longitude'], df5['Label']):
    label = folium.Popup(str(df5['Borough']) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)

KeyError: 'Latitude'

In [105]:
map_toronto